<a href="https://colab.research.google.com/github/suuuu0703/Analytics_Tips/blob/master/Colaboratory/Scrapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#前処理

##日本語フォントインストール（※実行後にランタイムの再起動が必要）

In [0]:
# 日本語フォントをダウンロードする。
!apt-get -y install fonts-ipafont-gothic

# キャッシュを削除する。
!rm /root/.cache/matplotlib/fontlist-v300.json # 消すべきcache

# ランタイムを再起動する。

##基本ライブラリインポート

In [0]:
try:
  %matplotlib inline
  import pandas as pd
  pd.options.display.max_columns = 1000
  pd.options.display.max_rows = 10000

  # 描画用ライブラリ
  import matplotlib.pyplot as plt
  import seaborn as sns
  sns.set(context='paper', style='whitegrid')
  sns.set_context("poster")
  fpath = {'fontname':'IPAGothic'}

  # 日本語化
  !pip install japanize-matplotlib
  import japanize_matplotlib

  clear_output()
  print('All Loaded')        
except Exception as e:
  print(e)
  pass

All Loaded


##TreasureData用ライブラリインポート

In [0]:
!pip install -U -q PyDrive td-client
from IPython.display import clear_output
try:

    # TD_API_KEY取得のためのGoogleドライブファイルインポート用ライブラリインポート
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from googleapiclient.discovery import build
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    drive_service = build('drive', 'v3')

    #Google Drive > Colab Notebooksディレクトリ配下にTD_API_KEYというファイル名でTDのAPI Key（Master Key）を記述して保管
    find_file = drive_service.files().list(q="name = 'TD_API_KEY'", fields="files(id)").execute()
    file_id = find_file.get('files', [])[0]['id']
    drive_file = drive.CreateFile({'id': file_id})
    td_api_key = drive_file.GetContentString().strip()

    # TD_REST_API用ライブラリインポート
    import tdclient
    def td_job_result(jobid):
      with tdclient.Client(apikey = td_api_key) as client:
          job_header = client.job(jobid).result_schema
          job_results = client.job_result(jobid)
      header = [row[0] for row in job_header]
      results = pd.DataFrame(job_results)
      results.columns = header
      return results
    clear_output()
    print('All Loaded')        
except Exception as e:
  print(e)
  pass

#データ取得

In [0]:
#jobidにTDの実行済みJOBのjobidを記述
df = td_job_result("jobid")